In [1]:
!pip install huggingface_hub

In [5]:
!pip install transformers datasets torch accelerate
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Use the GPT-Neo 1.3B model
model_name = "EleutherAI/gpt-neo-1.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
def chatgpt_neo_prompt(prompt, max_length=200, temperature=0.7):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [9]:
# manual set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

# generate chatbot responses
def chatbot_response(text):
    # Tokenize input text with padding and attention mask
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Move inputs to GPU
    #input_ids = inputs["input_ids"].to("cuda") #to cuda forces into GPU
    #attention_mask = inputs["attention_mask"].to("cuda")
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512).to(model.device)

    # Generate response from the model
    output = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=100,
        pad_token_id=tokenizer.pad_token_id,  # Use pad_token_id instead of eos_token_id
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7
    )

    # Decode and return the response
    return tokenizer.decode(output[0], skip_special_tokens=True)

# chat recursive loop
while True:
    user_input = input("Your input: ")

    if user_input.lower() == "exit":
        print("Chat terminated.")
        break

    #chatbot response
    bot_response = chatbot_response(user_input)
    print("Bot:", bot_response)


Your input: hi, how are you?
Bot: hi, how are you? Are you going to be here for a long time?"

"I'm not sure, but I hope so," the young man replied. "I have an idea that I might be going to be a farmer, but I'm not sure. I have a few things I need to do."

"That's good. I'm glad that you think that you'll be able to do something that you're not sure you can do. I have to tell


KeyboardInterrupt: Interrupted by user